# From LLMs to Minimal Agents
## Learning Goals
- Understand the agent paradigm: LLM = brain, tools = body.
- Build a minimal agent loop with two tools.
- Experience **decision → action → result** cycle.

This notebook connects with Section *1.3 From LLMs to LLM-based Agents* of the lecture notes.

## Step 1: Define the tools
Our agent can:
1. Use a **calculator** to evaluate math expressions.
2. Use a **dictionary** to define words.

> The agent (through its brain, an LLM) will decide which tool to use.

In [5]:
def calculator_tool(expression: str) -> str:
    try:
        return str(eval(expression))
    except:
        return 'Error in calculation'

dictionary = {
    'agent': 'An autonomous problem-solving system.',
    'database': 'An organized collection of structured information.'
}
def dictionary_tool(word: str) -> str:
    return dictionary.get(word.lower(), 'Word not found')

## Step 2: Initialize an LLM
We use HuggingFace via LangChain.  
⚠️ Replace with ChatOpenAI if using OpenAI API, or ChatOllama for local models.

In [6]:
# %load get_llm.py
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

# Load environment variables from .env
load_dotenv()

def get_llm(provider: str = "openai"):
    """
    Return a language model instance configured for either OpenAI or Ollama.

    This function centralizes the initialization of chat-based LLMs so that 
    notebooks and applications can switch seamlessly between cloud-based models 
    (OpenAI) and local models (Ollama).

    Parameters
    ----------
    provider : str, optional
        The backend provider to use. Options are:
        - "openai": returns a ChatOpenAI instance (requires OPENAI_API_KEY in .env).
        - "ollama": returns a ChatOllama instance (requires Ollama installed locally).
        Default is "openai".

    Returns
    -------
    langchain.chat_models.base.BaseChatModel
        A chat model instance that can be invoked with messages.

    Examples
    --------
    Initialize an OpenAI model (requires API key):

    >>> llm = get_llm("openai")
    >>> llm.invoke("Hello, how are you?")

    Initialize a local Ollama model (e.g., Gemma2 2B):

    >>> llm = get_llm("ollama")
    >>> llm.invoke("Summarize the benefits of reinforcement learning.")
    """
    if provider == "openai":
        return ChatOpenAI(
            model="gpt-4o-mini",  # can also be "gpt-4.1" or "gpt-4o"
            temperature=0
        )
    elif provider == "ollama":
        return ChatOllama(
            model="gemma2:2b",   # replace with any local model installed in Ollama
            temperature=0
        )
    else:
        raise ValueError("Unsupported provider. Use 'openai' or 'ollama'.")


## Step 3: Build the agent loop
- The **system prompt** explains available tools.
- The **LLM decides** which tool to use.
- The **agent executes** the tool and returns the result.

In [7]:
import json
from langchain.schema import HumanMessage, SystemMessage

def minimal_agent(query: str):
    llm = get_llm()
    
    system_prompt = """
    You are a minimal AI agent.
    Available tools:
    1. Calculator: for math expressions.
    2. Dictionary: for word definitions.

    Always respond with a valid JSON object in the format:
    {"tool": "calculator", "input": "2+2"}
    or
    {"tool": "dictionary", "input": "agent"}
    """
    
    decision = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=query)
    ])

    print("LLM decision:", decision.content)

    try:
        parsed = json.loads(decision.content)
        tool = parsed.get("tool", "").lower()
        argument = parsed.get("input", "")
    except Exception as e:
        return f"Could not parse decision: {e}"

    if tool == "calculator":
        return calculator_tool(argument)
    elif tool == "dictionary":
        return dictionary_tool(argument)
    else:
        return "No valid tool selected."


## Step 4: Run examples

In [8]:
print(minimal_agent('What is 15 * 3?'))
print(minimal_agent('Define: agent'))
print(minimal_agent('Define: database'))

LLM decision: {"tool": "calculator", "input": "15*3"}
45
LLM decision: {"tool": "dictionary", "input": "agent"}
An autonomous problem-solving system.
LLM decision: {"tool": "dictionary", "input": "database"}
An organized collection of structured information.


### Reflection
- The LLM functions as the agent’s brain, reasoning about the query and producing a structured JSON decision.

- The agent functions as the body, parsing the JSON and executing the selected tool with the provided input.

- This separation of concerns illustrates the foundation of tool-calling mechanisms later formalized in patterns such as ReAct and implemented in frameworks like LangGraph.

## Exercises

1. Add a new tool: a translator that translates English to Portuguese, and extend the system prompt to include it.

2. Modify the agent to loop until it returns a final JSON object with the key "answer" instead of only one decision.

3. Replace ChatOpenAI with ChatOllama in the get_llm() function to run the agent with a local model (e.g., gemma2:2b).